# From last lecture

Model parameters: $\Theta=(\pi,P,E)$ 

Observations: $X=(X_1, \ldots X_T)$

Hidden States: $Z=(Z_1, \ldots Z_T)$

- most probable path: $argmax_Z Pr(Z|X)$

Quiz:

1. How many state paths are there for $N=2$ states and sequences of length $T=100$ ?
2. Given $P(X,Z)$ how would you calculate $Pr(X)$ ?
3. Why would you need $P(X)$?

# $P(X)$: The Forward Algorithm

(See Notebook: HMM_001_Forward.ipynb for more details)

## Goal: avoid redundant calculations

... use the power of recursion ("dynamic programming")

$$P(X) = \sum_Z P(X_{1:T},Z_{1:T}) \to \sum_i P(X_{1:T},Z_T=i)$$

... if $P(X_{1:{t+1}},Z_{t+1})$ can be derived from $P(X_{1:{t}},Z_{t})$

## Idea: track all possibilities

We don't know any $Z_t$ $\longrightarrow$ Trellis graph.



<div>
   <img src="https://github.com/thomasmanke/ABS/raw/main/figures/HMM_HiddenTrellis.jpg",  width="1000">
</div>

- Store $\alpha_{ti} = Pr(X_{1:t}, Z_t=i)$ for each pvalue of $i$ (as $Z_t$ is unobsorved). This vector will be propagated forward in time.

- In our previous example we only had 2 states, but the graphic below is an illustration with 4 hidden states





## Graphical Summary:  2 Steps

<div>
   <img src="https://github.com/thomasmanke/ABS/raw/main/figures/HMM_Forward.jpg",  width="800">
</div>


<div>
   <img src="https://github.com/thomasmanke/ABS/raw/main/figures/HMM_Forward_summary.jpg",  width="800">
</div>

- The Markov Model pushes the state $Z$ forward in time $\longrightarrow$ *matrix multiplication* with $P$
- Emission probabilities: take into account the state-specifc probabilities for observation $X_t$ $\longrightarrow$ *element-wise multiplication* with proper column of $E$


## Notice
- **Marginalization:** $Pr(X_{1:t}) = \sum_i Pr(X_{1:t}, Z_t = i) = \sum_i \alpha_{ti}$.  
- **Recursion Efficiency:** Calculation of $Pr(X)$ requires $T N^2$ calculations $\ll N^T$ 
    - Example: $(N,T) = (2, 100) \longrightarrow 400 \ll 2^{100}$  
- **Emission matrix** $E_{ik}$ serves as lookup table for given observation $X_t=k$ at time $t$. ($k=f(t)$)

## A single step forward

Starting for a given $\alpha_{ti}$ th code below illustrates a single update in time. All subsequent times follow the same update, but the observed value of $x_t$ will geenerally change from time to time

In [ ]:
import numpy as np
pi=np.array( [0.75, 0.25] )                          # initial state probability
P =np.array([ [0.8, 0.2], [0.1, 0.9] ])              # transition probabilites
E =np.array([ [0.7, 0.2, 0.1], [0.1, 0.1, 0.8] ])    # emission probabilties

alpha = np.array([0.75, 0.25])  # P(Z_t-1=i|X_t-1):  initial probability at time t-1
xt = 1                          # given observation at time t (e.g. 1)
 
alpha = alpha.dot(P)            # P(Z_t=i | X_t-1):  push Z from t-1 --> t (state transition)
print('after state transition: ', alpha) 

ep=E[:,xt]                      # P(X_t=xt|Z_t=i):  emission probabilities of for each state
print('emission vector:        ', LH)

alpha = ep * alpha              # P(Z_t=i | X_1:t): posterior  (take into account new observation X_t)
print('new probability         ', alpha) 

after state transition:  [0.625 0.375]
emission vector:         [0.2 0.1]
new probability          [0.125  0.0375]


## Extensions

- Above we propagated $Pr(X_{1:t}, Z_t=i)$ forward in time, but this works similarly for $Pr(Z_t=i| X_{1:t})$ (plus extra normaliztion)
- Forward-backward algortihm for another interesting quantity:  $Pr(Z_t=i| X_{1:T})$


<div>
   <img src="https://github.com/thomasmanke/ABS/raw/main/figures/HMM_ForwardBackward.jpg",  width="1200">
</div>

$$
Pr(Z_t = i | X_{1:T}) = \gamma_{ti} = \frac{\alpha_{ti} \beta_{ti}}{\sum_k \alpha_{tk} \beta_{tk}} 
$$



# The good news: There is software

In [ ]:
#%%script echo install only once
!pip install hmmlearn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 129 kB 4.1 MB/s 


## HMMlearn: Generating Sequences and Observations

In [ ]:
from hmmlearn import hmm

pi=np.array( [0.75, 0.25] )                          # initial state probability
P =np.array([ [0.8, 0.2], [0.1, 0.9] ])              # transition probabilites
E =np.array([ [0.7, 0.2, 0.1], [0.1, 0.1, 0.8] ])    # emission probabilties

np.random.seed(42)                       # only for reproducibiltiy

# define HMM and set parameters
model = hmm.MultinomialHMM(n_components=2)
model.startprob_ = pi                    # initial state prob
model.transmat_  = P                     # transition prob
model.emissionprob_ = E                  # emission prob

# generate sequence
X,Z = model.sample(50)                 # c.f. Z, X = generate_HMM(P,pi,E)
print('states Z       =',*Z.flatten()) # Z.shape = (T,)
print('observations X =',*X.flatten()) # X.shape = (T,1)

# Group Task (30 min): HMM Generation

1. Make up your own hidden Markov story, draw the corresponding state graph, and define the Hidden Markov Model. 
  - Please keep it simple; less than 5 hidden states and less than 5 possible observations. 
  - Also make sure that the Markov Model for the hidden states is *ergodic* (what was that?)

2. Choose your own emission probabilties, transition probabilties and the initital state distribution - make sure they correspond to probabilties. 

3. Simulate $T=1000$ steps.

4. Record (only) the sequence of observations that were generated and store the results as string in a text file (for latter use). Be kind and use integer encoding of observations, i.e. $0,1,\ldots$ regardless of the interpretation.

5. Share your story, code, results and report back to the class.

In [ ]:
%%script echo edit before execution (only one per group)

model = hmm.MultinomialHMM( ... )
model.startprob_ = ...                   # initial state prob
model.transmat_  = ...                     # transition prob
model.emissionprob_ = ...                  # emission prob

T=1000
X,Z = ... sample observations and hidden states from model

fn='obs_group1.txt'           # choose a group-specific filename

### no need to edit below ####

# write ######
with open(fn, 'w') as f:
  m = map(str, X.flatten())   # flatten array and convert numbers to strings
  f.write(' '.join(list(m))) 

# read (for later) ######
with open(fn, "r") as f:
  line  = f.readline().split()  # read first line and split

Xr = list(map(np.int64, line))   # map line to np.int64
Xr = np.array(Xr).reshape(-1,1)  # return desired shape (T, 1)

#print('X =',*X[:10])
#print('Xr=',*Xr[:10])
np.all(X==Xr)

... continue withh HMM_002_Viterbi (Section HMMlearn)